In [1]:
device = 'cuda:3'

In [8]:
from data_loader import get_mix_instruct

prompts, references, ds_name = get_mix_instruct("train", 21000)
prompts_val, references_val, ds_name_valid = get_mix_instruct("validation", 5000)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅
Dataset: mix-instruct_validation_5000 found in cache, loading from cache ✅


In [9]:
from utility_functions.delift_se import get_delift_se_utility

utility, utility_name = get_delift_se_utility(prompts, references, ds_name)

Utility: mix-instruct_train_21000_delift-se found in cache, loading from cache ✅


In [10]:
from subset import create_subset, get_subset

subset, subset_name = create_subset(utility, utility_name)

Subset: mix-instruct_train_21000_delift-se_0.3 found in cache, loading from cache ✅


In [14]:
from inference import generate_responses
from evaluation import compute_metrics
# For LLama 3.2 (initial)
# Reported BGE: 0.73, ROUGE: 0.28
responses_llama, generation_name_llama = generate_responses(prompts_val, 'meta-llama/Llama-3.2-3B', ds_name_valid, device, batch_size=400)
compute_metrics(responses_llama, references_val, generation_name_llama, device=device)

Generation: mix-instruct_validation_5000_Llama-3.2-3B_100 found in cache, loading from cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_100_bge not found in cache, computing now 🏃


Computing BGE: 100%|██████████| 10/10 [00:49<00:00,  4.95s/it]


Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_100_bge computed and saved to cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_100_rouge not found in cache, computing now 🏃


Evaluating ROUGE: 100%|██████████| 2/2 [00:14<00:00,  7.44s/it]

Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_100_rouge computed and saved to cache ✅


{'bge': 0.70930254, 'rouge': 0.20283267710318825}

Before correction: {'bge': 0.70851624, 'rouge': 0.2012717129499011}

In [16]:
# For LLama 3.2 (finetuned on 30% data selected with DeLiftSE)
# Reported BGE: 0.85, ROUGE: 0.55
from finetune import fine_tune_model
from data_loader import get_mix_instruct
from utility_functions.delift_se import get_delift_se_utility
from subset import create_subset, get_subset
import importlib
import inference
importlib.reload(inference)
from inference import generate_responses
from evaluation import compute_metrics



prompts, references, ds_name = get_mix_instruct("train", 21000)
utility, utility_name = get_delift_se_utility(prompts, references, ds_name)
subset, subset_name = create_subset(utility, utility_name)
s_prompts, s_references = get_subset(subset, prompts, references)
prompts_val, references_val, ds_name_valid = get_mix_instruct("validation", 5000)
model_dir = fine_tune_model('meta-llama/Llama-3.2-3B', prompts, references, prompts_val, references_val, subset_name)

responses_llama_ft, generation_name_llama_ft = generate_responses(prompts_val, model_dir, ds_name_valid, device, batch_size=400, max_length=150)
compute_metrics(responses_llama_ft, references_val, generation_name_llama_ft, device=device)

Dataset: mix-instruct_train_21000 found in cache, loading from cache ✅
Utility: mix-instruct_train_21000_delift-se found in cache, loading from cache ✅
Subset: mix-instruct_train_21000_delift-se_0.3 found in cache, loading from cache ✅
Dataset: mix-instruct_validation_5000 found in cache, loading from cache ✅
Finetune: Fine-tuned model found in cache. Skipping Training ✅
Generation: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150 not found in cache, generating responses 🏃


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Generating responses: 100%|██████████| 13/13 [04:36<00:00, 21.30s/it]


Generation: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150 generated and saved to cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150_bge not found in cache, computing now 🏃


Computing BGE: 100%|██████████| 10/10 [00:57<00:00,  5.77s/it]


Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150_bge computed and saved to cache ✅
Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150_rouge not found in cache, computing now 🏃


Evaluating ROUGE: 100%|██████████| 2/2 [00:19<00:00,  9.95s/it]

Evaluate: mix-instruct_validation_5000_Llama-3.2-3B_mix-instruct_train_21000_delift-se_0.3_150_rouge computed and saved to cache ✅


{'bge': 0.7600144, 'rouge': 0.2821869937004374}